In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
#spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar > /dev/null   

--2022-05-14 23:08:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.90MB/s    in 0.2s    

2022-05-14 23:08:24 (4.90 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://amazon-reviews.cp9vtji419xj.us-east-2.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [5]:
df = spark.read.format("jdbc").option("url",jdbc_url).option("dbtable", "vine_table").option("user","postgres").option("password",password).load()

In [9]:
df.printSchema()

ConnectionRefusedError: ignored

In [6]:
df_filtered = df[df.total_votes > 20]
#df_filtered.show()

In [7]:
div_df = df_filtered[(df_filtered.helpful_votes/df_filtered.total_votes) >= 0.5]
print(div_df.count())
#div_df.show()

74750


In [8]:
vine_program_df = div_df[div_df.verified_purchase == 'Y']
print(vine_program_df.count())
#vine_program_df.show()

46282


In [9]:
not_vine_program_df = div_df[div_df.verified_purchase == 'N']
print(not_vine_program_df.count())
#not_vine_program_df.show()

28468


In [10]:
five_star_paid = vine_program_df[vine_program_df.star_rating == 5].count()
not_five_star_paid = vine_program_df[vine_program_df.star_rating != 5].count()
percentage = (five_star_paid / not_five_star_paid) * 100
print('Number of 5 stars: ' + str(five_star_paid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 22393 Percentage: 93.74


In [11]:
five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating == 5].count()
not_five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating != 5].count()
percentage = (five_star_unpaid / not_five_star_unpaid) * 100
print('Number of 5 stars: ' + str(five_star_unpaid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 12366 Percentage: 76.8
